In [40]:
def remove_tokens(x, tok2remove):
    return ' '.join(['' if t in tok2remove else t for t in x.split(' ')])

In [43]:
remove_tokens(data1[data1['transcript'].str.contains('airdynamisis')].transcript.values[0],{})

'know best human power flight one thing get going sort things working youngster interest model airplane ornithopter autogyro helicopter glider power plane indoor model outdoor model everything thought lot fun wonder people share enthusiasm navy pilot training college get sailplane flying power plane flying consider sailplane sort hobby fun get tangle great professor type convince everybody else field good way get really deep science going field weather modification although getting ph aeronautics weather modification subject getting start graduate student could go around various talks given hitchhiker ride east coast everybody would talk professional field hate communicate result get absolutely unique background field start company research weather modification anybody lot things go start aerovironment employee one two three sort fumble along try get interest project airdynamisis like want work aerospace company big many year project small project company slowly grow thing excite sudde

In [9]:
!sudo apt update -y
!pip install nltk
!pip install gensim
!pip install spacy
!pip install pyLDAvis
# !sudo apt-get install default-jre -y

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease               
Hit:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease             
Reading package lists... Done
Building dependency tree       
Reading state information... Done
49 packages can be upgraded. Run 'apt list --upgradable' to see them.
     |████████████████████████████████| 1.6MB 1.7MB/s eta 0:00:01
     |████████████████████████████████| 235kB 3.0MB/s eta 0:00:01
     |████████████████████████████████| 829kB 4.6MB/s eta 0:00:01
     |████████████████████████████████| 92kB 5.2MB/s eta 0:00:01


  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=0f2caa5b21f947f7c8558c26f74aa5b2d95251f107d080473b1c1afaea79b28a
  Stored in directory: /home/jovyan/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491057 sha256=08ac082fc575caecadb8b8cd767029cb9466cad91fadf803c3a3d97b02871a94
  Stored in directory: /home/jovyan/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built pyLDAvis future


In [10]:
import nltk
import pickle

import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

import os
from os import path

In [19]:
with open(path.expanduser('~/work/data/data.pkl'), "rb") as input_file:
    data1 = pickle.load(input_file)

In [12]:
data1[:5]

,transcript,url,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,views
0,good morning great blow away whole thing fact ...,https://www.ted.com/talks/ken_robinson_says_sc...,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,1140825600,60,Ken Robinson,Ken Robinson: Do schools kill creativity?,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...","[{'id': 865, 'hero': 'https://pe.tedcdn.com/im...",Author/educator,"['children', 'creativity', 'culture', 'dance',...",Do schools kill creativity?,47227110
1,thank much chris truly great honor opportunity...,https://www.ted.com/talks/al_gore_on_averting_...,265,With the same humor and humanity he exuded in ...,977,TED2006,1140825600,43,Al Gore,Al Gore: Averting the climate crisis,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 544}, {'i...","[{'id': 243, 'hero': 'https://pe.tedcdn.com/im...",Climate advocate,"['alternative energy', 'cars', 'climate change...",Averting the climate crisis,3200520
2,hello voice mail old friend call tech support ...,https://www.ted.com/talks/david_pogue_says_sim...,124,New York Times columnist David Pogue takes aim...,1286,TED2006,1140739200,26,David Pogue,David Pogue: Simplicity sells,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 964}, {'i...","[{'id': 1725, 'hero': 'https://pe.tedcdn.com/i...",Technology columnist,"['computers', 'entertainment', 'interface desi...",Simplicity sells,1636292
3,today happy hear sustainable development save ...,https://www.ted.com/talks/majora_carter_s_tale...,200,"In an emotionally charged talk, MacArthur-winn...",1116,TED2006,1140912000,35,Majora Carter,Majora Carter: Greening the ghetto,1,1151367060,"[{'id': 3, 'name': 'Courageous', 'count': 760}...","[{'id': 1041, 'hero': 'https://pe.tedcdn.com/i...",Activist for environmental justice,"['MacArthur grant', 'activism', 'business', 'c...",Greening the ghetto,1697550
4,years ago take task teach global development s...,https://www.ted.com/talks/hans_rosling_shows_t...,593,You've never seen data presented like this. Wi...,1190,TED2006,1140566400,48,Hans Rosling,Hans Rosling: The best stats you've ever seen,1,1151440680,"[{'id': 9, 'name': 'Ingenious', 'count': 3202}...","[{'id': 2056, 'hero': 'https://pe.tedcdn.com/i...",Global health expert; data visionary,"['Africa', 'Asia', 'Google', 'demo', 'economic...",The best stats you've ever seen,12005869


In [13]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data1['transcript']))

print(data_words[:1])

[['good', 'morning', 'great', 'blow', 'away', 'whole', 'thing', 'fact', 'leaving', 'three', 'theme', 'running', 'conference', 'relevant', 'want', 'talk', 'one', 'extraordinary', 'evidence', 'human', 'creativity', 'presentation', 'people', 'variety', 'range', 'second', 'put', 'us', 'place', 'idea', 'going', 'happen', 'terms', 'future', 'idea', 'may', 'play', 'interest', 'education', 'actually', 'find', 'everybody', 'interest', 'education', 'find', 'interest', 'dinner', 'party', 'say', 'work', 'education', 'actually', 'often', 'dinner', 'party', 'frankly', 'work', 'education', 'ask', 'never', 'ask', 'back', 'curiously', 'strange', 'say', 'somebody', 'know', 'say', 'say', 'work', 'education', 'see', 'blood', 'run', 'face', 'like', 'oh', 'god', 'know', 'one', 'night', 'week', 'ask', 'education', 'pin', 'wall', 'one', 'things', 'go', 'deep', 'people', 'right', 'like', 'religion', 'money', 'things', 'big', 'interest', 'education', 'think', 'huge', 'vest', 'interest', 'partly', 'education', '

In [14]:
# Create Dictionary
id2word = corpora.Dictionary(data_words)

# Create Corpus
texts = data_words

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 2), (1, 1), (2, 4), (3, 1), (4, 1), (5, 6), (6, 2), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 3), (24, 1), (25, 1), (26, 1), (27, 2), (28, 2), (29, 3), (30, 3), (31, 2), (32, 3), (33, 2), (34, 3), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 2), (42, 1), (43, 3), (44, 5), (45, 1), (46, 1), (47, 1), (48, 2), (49, 2), (50, 1), (51, 1), (52, 2), (53, 1), (54, 1), (55, 1), (56, 1), (57, 3), (58, 1), (59, 1), (60, 1), (61, 1), (62, 3), (63, 1), (64, 3), (65, 1), (66, 1), (67, 5), (68, 3), (69, 1), (70, 1), (71, 2), (72, 1), (73, 3), (74, 1), (75, 1), (76, 6), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 9), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 12), (95, 1), (96, 1), (97, 2), (98, 1), (99, 1), (100, 2), (101, 2), (102, 1), (103, 2), (104, 1), (105, 1), (106, 1), (107, 1), (108, 2), (109, 1), (110, 3

['ability',
 'abstract',
 'academic',
 'accord',
 'achievement',
 'actually',
 'adhd',
 'adopt',
 'adult',
 'advice',
 'affection',
 'afford',
 'afterward',
 'agent',
 'ago',
 'agree',
 'al',
 'alien',
 'allow',
 'along',
 'always',
 'america',
 'andrew',
 'angeles',
 'anniversary',
 'annoy',
 'annoying',
 'another',
 'anything',
 'anyway',
 'around',
 'art',
 'artist',
 'arts',
 'ask',
 'attention',
 'audition',
 'available',
 'avert',
 'avon',
 'aware',
 'away',
 'ba',
 'back',
 'ballet',
 'base',
 'bearing',
 'beat',
 'become',
 'bed',
 'beginning',
 'being',
 'believe',
 'beneath',
 'benign',
 'benignly',
 'better',
 'big',
 'bit',
 'blood',
 'blow',
 'board',
 'body',
 'book',
 'born',
 'bottom',
 'box',
 'boy',
 'brain',
 'break',
 'brilliant',
 'bring',
 'brownie',
 'call',
 'callosum',
 'calm',
 'capacity',
 'career',
 'careful',
 'carry',
 'carson',
 'cat',
 'ceiling',
 'celebrate',
 'century',
 'chair',
 'chance',
 'chestnut',
 'child',
 'childhood',
 'choreographer',
 'class

In [7]:
with open(path.expanduser('~/work/data/vocabulary.pkl'), 'wb') as output:
    pickle.dump(np.array(list(id2word.values()), dtype=str), output, pickle.HIGHEST_PROTOCOL)

In [9]:
mallet_path = path.expanduser('~/work/jupyter_notebooks/notebooks/mallet-2.0.8/bin/mallet') # update this path
prefix = path.expanduser('~/work/jupyter_notebooks/notebooks/mallet-dep/')
model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=35, id2word=id2word, random_seed = 1, prefix=prefix)
# Show Topics
pprint(model.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=model, texts=data_words, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

[(27,
  [('universe', 0.022666898924356575),
   ('light', 0.016918521099159516),
   ('theory', 0.011866916343683316),
   ('space', 0.011692723076253103),
   ('science', 0.01077820842224448),
   ('number', 0.009036275747942342),
   ('physics', 0.008796760005225798),
   ('galaxy', 0.008753211688368245),
   ('particle', 0.008687889213081914),
   ('star', 0.008513695945651701)]),
 (14,
  [('war', 0.0310866761691441),
   ('people', 0.012421095499898187),
   ('conflict', 0.009819226679336636),
   ('kill', 0.009819226679336636),
   ('military', 0.009230978076427069),
   ('refugee', 0.008348605172062717),
   ('world', 0.007828231407950406),
   ('peace', 0.0076924817303558905),
   ('violence', 0.007488857213964117),
   ('force', 0.007330482590103849)]),
 (24,
  [('city', 0.06589404731911774),
   ('place', 0.02272050571448203),
   ('people', 0.021048533015734866),
   ('building', 0.016857149401067313),
   ('street', 0.015574540207507844),
   ('space', 0.013994182451157784),
   ('york', 0.0112457

In [10]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word, random_seed = 1)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
        print('No. Topics ', num_topics,' - Coherence ',coherencemodel.get_coherence())

    return model_list, coherence_values

limit=101; start=10; step=10;
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_words, start=start, limit=limit, step=step)
# Show graph

x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

No. Topics  10  - Coherence  0.3859702030700771
No. Topics  20  - Coherence  0.45740016941974015
No. Topics  30  - Coherence  0.45953520248675267
No. Topics  40  - Coherence  0.461542256045851
No. Topics  50  - Coherence  0.47737658463317856


KeyboardInterrupt: 

In [10]:
for i in np.arange(35):
    print('Topic', i, '\n' ,model.print_topic(i, topn=10),'\n')

Topic 0 
 0.030*"energy" + 0.028*"water" + 0.014*"oil" + 0.014*"climate" + 0.011*"carbon" + 0.011*"power" + 0.011*"make" + 0.011*"percent" + 0.010*"fuel" + 0.010*"problem" 

Topic 1 
 0.040*"sound" + 0.039*"play" + 0.033*"music" + 0.021*"hear" + 0.015*"song" + 0.015*"voice" + 0.010*"make" + 0.010*"call" + 0.010*"playing" + 0.009*"listen" 

Topic 2 
 0.029*"company" + 0.028*"dollar" + 0.028*"money" + 0.025*"business" + 0.015*"pay" + 0.015*"market" + 0.013*"work" + 0.012*"make" + 0.012*"buy" + 0.011*"job" 

Topic 3 
 0.023*"day" + 0.022*"back" + 0.021*"time" + 0.014*"start" + 0.011*"walk" + 0.009*"night" + 0.009*"put" + 0.008*"morning" + 0.008*"work" + 0.008*"hand" 

Topic 4 
 0.031*"health" + 0.019*"disease" + 0.016*"care" + 0.016*"drug" + 0.014*"people" + 0.013*"doctor" + 0.011*"patient" + 0.010*"percent" + 0.010*"hospital" + 0.010*"medical" 

Topic 5 
 0.026*"ocean" + 0.016*"fish" + 0.016*"sea" + 0.015*"water" + 0.010*"ice" + 0.009*"back" + 0.008*"coral" + 0.007*"place" + 0.007*"time"

In [85]:
limit=52; start=33; step=1;
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_words_bigrams, start=start, limit=limit, step=step)
# Show graph

x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

KeyboardInterrupt: 

In [13]:
with open(path.expanduser('~/work/test/lda/ldamallet.pkl'), 'wb') as output:
    model.save(output)

In [11]:
with open(path.expanduser('~/work/test/lda/lda.pkl'), 'wb') as output:
    pickle.dump(model, output, pickle.HIGHEST_PROTOCOL)

In [12]:
def format_topics_sentences(ldamodel, corpus=corpus, texts=data1):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=model, corpus=corpus, texts=data1['transcript'].values)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,18.0,0.2056,"school, kid, student, education, teacher, lear...",good morning great blow away whole thing fact ...
1,1,0.0,0.1856,"energy, water, oil, climate, carbon, power, ma...",thank much chris truly great honor opportunity...
2,2,23.0,0.1607,"technology, computer, machine, system, human, ...",hello voice mail old friend call tech support ...
3,3,24.0,0.2459,"city, place, people, building, street, space, ...",today happy hear sustainable development save ...
4,4,8.0,0.5551,"country, world, africa, percent, china, state,...",years ago take task teach global development s...
5,5,25.0,0.1752,"life, story, world, experience, god, feel, mom...",thank tell challenge excite excitement get cha...
6,6,3.0,0.2372,"day, back, time, start, walk, night, put, morn...",september morning seventh birthday come downst...
7,7,6.0,0.3210,"design, make, building, material, create, buil...",going present three project rapid fire much ti...
8,8,19.0,0.1220,"book, write, language, read, word, words, stor...",wonderful back love wonderful gathering must w...
9,9,25.0,0.2059,"life, story, world, experience, god, feel, mom...",often ask surprise book say get write would ne...


In [13]:
with open(path.expanduser('~/work/test/lda/docdominanttopic.pkl'), 'wb') as output:
    pickle.dump(df_dominant_topic, output, pickle.HIGHEST_PROTOCOL)

In [14]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet

,Topic_Num,Topic_Perc_Contrib,Keywords,Text
0,0.0,0.5946,"energy, water, oil, climate, carbon, power, ma...",well big announcement make today really excite...
1,1.0,0.5598,"sound, play, music, hear, song, voice, make, c...",adam ockelford promise much talking lot derek ...
2,2.0,0.4428,"company, dollar, money, business, pay, market,...",want talk social innovation social happen trip...
3,3.0,0.5899,"day, back, time, start, walk, night, put, morn...",brain magic brain magic brain magic indicate a...
4,4.0,0.5385,"health, disease, care, drug, people, doctor, p...",one first patient see pediatrician sol beautif...
5,5.0,0.5609,"ocean, fish, sea, water, ice, back, coral, pla...",fascinate lifetime beauty form function giant ...
6,6.0,0.4658,"design, make, building, material, create, buil...",architect often ask origin form design kind fo...
7,7.0,0.4784,"love, feel, life, fear, hard, make, learn, fee...",age three hundred seventy two think deep regre...
8,8.0,0.6245,"country, world, africa, percent, china, state,...",nice tonight working history income wealth dis...
9,9.0,0.5664,"brain, memory, mind, neuron, control, body, ch...",well chris point study human brain function st...


In [15]:
with open(path.expanduser('~/work/test/lda/topicdominantdoc.pkl'), 'wb') as output:
    pickle.dump(sent_topics_sorteddf_mallet, output, pickle.HIGHEST_PROTOCOL)

In [16]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics

,Dominant_Topic,Topic_Keywords,Num_Documents,Perc_Documents
0.0,18.0,"school, kid, student, education, teacher, lear...",82.0,0.0334
1.0,0.0,"energy, water, oil, climate, carbon, power, ma...",90.0,0.0366
2.0,23.0,"technology, computer, machine, system, human, ...",75.0,0.0305
3.0,24.0,"city, place, people, building, street, space, ...",88.0,0.0358
4.0,8.0,"country, world, africa, percent, china, state,...",87.0,0.0354
...,...,...,...,...
2452.0,32.0,"child, family, life, mother, years, father, ba...",NaN,NaN
2453.0,15.0,"earth, planet, years, life, space, mars, syste...",NaN,NaN
2454.0,33.0,"human, idea, problem, kind, social, question, ...",NaN,NaN
2455.0,12.0,"people, political, power, american, democracy,...",NaN,NaN


In [17]:
with open(path.expanduser('~/work/test/lda/topicdist.pkl'), 'wb') as output:
    pickle.dump(df_dominant_topics, output, pickle.HIGHEST_PROTOCOL)

In [18]:
sent_topics_sorteddf_mallet.to_csv(path.expanduser('~/work/test/lda/topics.csv'))

'/home/jovyan/work/jupyter_notebooks/notebooks/mallet-dep/'